In [ ]:
# !pip install pyodbc
# !pip install pandas
# !pip install azure-storage-blob
# !pip install pysftp
# !pip install paramiko
# !pip install python-dotenv
# !pip install pytz
# !pip install databricks-sql-connector
# !pip install azure-keyvault-secrets azure-identity

In [4]:
import paramiko
import os
from dotenv import load_dotenv
from databricks import sql
from azure.storage.blob import BlobServiceClient
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential
from io import StringIO, BytesIO
import pandas as pd
from datetime import datetime
import logging
import pysftp
import pytz

In [16]:
load_dotenv()

True

In [8]:
SFTP_HOST = "deltaintegrationstorage.blob.core.windows.net"
SFTP_USERNAME = "deltaintegrationstorage.mjuser"  # storageaccount.username format
SSH_PRIVATE_KEY_PATH = r"C:\Users\MarvinJayReyes\Documents\DeltaInnovation\grappler-transfer\grappler-integration\mjuser_test"  # Raw string for Windows paths

In [9]:
def get_current_date():
    nz_tz = pytz.timezone('Pacific/Auckland')
    nz_time = datetime.now(nz_tz)
    return nz_time.strftime("%d-%m-%Y")

In [10]:
def fetch_data_from_databricks(query, connection):
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            columns = [desc[0] for desc in cursor.description]
            result = cursor.fetchall()
        return pd.DataFrame(result, columns=columns)
    except Exception as e:
        logger.error(f"Error fetching data from Databricks: {str(e)}", exc_info=True)
        raise

In [14]:
query = f"""  SELECT * FROM delta_datawarehouse_merger.deltadbprodnz.policy WHERE id = 1 """
try:
    connection = sql.connect(
    server_hostname=os.getenv('DATABRICKS_SERVER_HOSTNAME'),
    http_path=os.getenv('DATABRICKS_HTTP_PATH'),
    access_token=os.getenv('DATABRICKS_TOKEN')
    )
    df = fetch_data_from_databricks(query, connection)
finally:
    connection.close()

AttributeError: 'function' object has no attribute 'close'

In [20]:
current_date = get_current_date()
blob_name = f'grapplerau_delta_{current_date}.csv'
df['file_name'] = blob_name
df.to_csv(f'grapplerau_delta_{current_date}.csv', index=False)

NameError: name 'df' is not defined

In [15]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    # Option 1: Load private key from file
private_key = paramiko.RSAKey.from_private_key(secret.value)

AttributeError: 'str' object has no attribute 'readlines'

In [37]:
ssh.connect(SFTP_HOST, username=SFTP_USERNAME, pkey=private_key)
    
sftp = ssh.open_sftp()
sftp.put(f'grapplerau_delta_{current_date}.csv', f'grapplerau_delta_{current_date}.csv')
sftp.close()
ssh.close()

# Clean up local file
os.remove(f'grapplerau_delta_{current_date}.csv')